In [1]:
import pixeltable as pxt
from pixeltable.functions.video import make_video

CACHE = "cache_cd64"
audio_chunks_view = pxt.get_table(f"{CACHE}.table_audio_chunks")
frames_view = pxt.get_table(f"{CACHE}.table_frames")

Connected to Pixeltable database at: postgresql+psycopg://postgres:@/pixeltable?host=/Users/moteroperdido/.pixeltable/pgdata


Error: Directory 'cache_cd64' does not exist.

In [5]:
frames_view.columns

['pos',
 'frame_idx',
 'pos_msec',
 'pos_frame',
 'frame',
 'im_caption',
 'video',
 'audio_extract']

In [72]:
################
# MCP Tools
################
from typing import List


def get_clip_by_speech_sim(user_query: str, top_k: int = 3) -> List[pxt.DataFrame]:
    """Get a video clip based on the user query.

    Args:
        user_query: The user query to search for.

    Returns:
        The path to the video clip.
    """

    sims = audio_chunks_view.chunk_text.similarity(user_query)
    results = audio_chunks_view.select(
        audio_chunks_view.pos,
        audio_chunks_view.start_time_sec,
        audio_chunks_view.end_time_sec,
        similarity=sims,
    ).order_by(sims, asc=False)

    video_clips = []
    top_k_entries = results.limit(top_k).collect()
    if len(top_k_entries) > 0:
        for entry in top_k_entries:
            start_time_sec = float(entry["start_time_sec"])
            end_time_sec = float(entry["end_time_sec"])

            sampled_clip = frames_view.select(make_video(frames_view.pos, frames_view.frame)).where(
                (frames_view.pos_msec >= start_time_sec * 1e3) & (frames_view.pos_msec <= end_time_sec * 1e3)
            )
            video_clips.append(sampled_clip)
    return video_clips


def get_clip_by_caption_sim(user_query: str, top_k: int = 3) -> List[pxt.DataFrame]:
    """Get a video clip based on the user query using caption similarity.

    Args:
        video_name: The name of the video index.
        user_query: The user query to search for.
        top_k: The number of top results to return.

    Returns:
        A string listing the paths to the video clips.
    """
    sims = frames_view.im_caption.similarity(user_query)
    results = frames_view.select(
        frames_view.pos_msec,
        frames_view.im_caption,
        similarity=sims,
    ).order_by(sims, asc=False)

    video_clips = []
    N = 10
    top_k_entries = results.limit(top_k).collect()
    if len(top_k_entries) > 0:
        for entry in top_k_entries:
            pos_msec = float(entry["pos_msec"])
            sampled_clip = frames_view.select(make_video(frames_view.pos, frames_view.frame)).where(
                (frames_view.pos_msec >= pos_msec - N * 1000) & (frames_view.pos_msec <= pos_msec + N * 1000)
            )
            video_clips.append(sampled_clip)
    return video_clips


In [ ]:
# Dummy sampling from both indexes (text based, audio transcripts and image captions)
by_speech = get_clip_by_speech_sim("goal", top_k=1)
by_caption = get_clip_by_caption_sim("Fernando Hierro sitting on a bench", top_k=1)

In [ ]:
# Calling collect to materialize the results
clip_by_speech = by_speech[0].collect()
clip_by_caption = by_caption[0].collect()

In [75]:
frames_view.select(
    frames_view.im_caption,
    frames_view.pos_msec,
).show(20)

im_caption,pos_msec
"The image is a screenshot from a FIFA video game, specifically from the ""FIFA 19"" game mode. It displays the starting lineup for Portugal's national football team for the 2018 FIFA World Cup. The screen shows the names of the players listed from top to bottom as follows: 1. Rui Patrício 2. Pepe 3. Raphael Guerreiro 4. José Fonte 5. Cristiano Ronaldo 6. João Moutinho 7. Bernardo Silva 8. William 9. Bruno Fernandes 10. Gonçalo Guedes 11. Cedric Below the player list, there is a small map showing the formation of the team, which is a 4-3-3. The coach of the team is Fernando Santos. The background features a stadium with a large crowd, indicating that this is a match scenario. The overall theme and design are consistent with the FIFA 19 game interface, including the use of the FIFA logo and the game's branding.",0.
"The image is a screenshot from a FIFA video game, specifically showing the starting lineup for Portugal in a match. The interface displays various elements typical of a soccer video game: 1. **Team Information**: At the top left corner, there is a red and white flag with the text ""PORTUGAL"" indicating the team's nationality. 2. **Substitutes List**: Below the team information, there is a list of substitutes. The list includes: - Anthony Lopes (Sub) - Beto (Sub) - Bruno Alves (Sub) ...... - 6: Conte - 18: Bute - 14: Killa - 21: Lérior - 1: Patri 4. **Game Mode**: In the bottom right corner, there is a small icon that indicates the game mode or setting, which is not clearly visible in this screenshot. This setup is typical of a soccer video game where players can customize their teams and view detailed information about the players on the field. The interface provides a comprehensive overview of the team's structure and the players' positions during the match.",2000.
"The image is a screenshot from the video game FIFA 19, showing the starting lineup for Portugal in a match. The interface displays various players and their positions on the field. Here is a detailed breakdown of the information presented: ### Starting Lineup: - **Portugal** - **Substitutes:** - Anthony Lopes (7) - Beto (22) - Bruno Alves (15) - Manuel Fernandes (16) - André Silva (17) - João Mario (10) - Ruben Dias (15) - Ricardo (15) - Gelson Martins ...... 15) - Right Midfielder - **Gelson Martins** (19) - Left Winger - **Mario Rui** (19) - Center Forward - **Ricardo Quaresma** (23) - Right Forward ### Additional Information: - **Rúben Patrício** - Substituted by Cédric (21) - **Rúmã Patrício** - Substituted by Cédric (21) This lineup represents the starting eleven for Portugal in the game, with the substitutes listed below them. The interface also includes the FIFA logo in the top right corner, indicating that this is an official FIFA game.",4000.
"The image is a screenshot from the FIFA 19 video game, showing the starting lineup for Portugal in a match. The interface displays various players and their positions on the field. Here is a detailed breakdown of the information presented: ### Starting Lineup: - **Goalkeeper:** Rúben Patrício (Position: 1) - **Defenders:** - **Left Back:** Rúben Dias (Position: 4) - **Right Back:** William (Position: 21) - **Center Back:** Gelson Martins (Position: 5) - **Center Back:** Pepe (Positi ...... Position: 11) - **Central Midfielder:** Gelson Martins (Position: 5) - **Central Midfielder:** William (Position: 21) - **Central Midfielder:** Rúben Dias (Position: 4) - **Central Midfielder:** Ricardo (Position: 15) - **Central Midfielder:** Gelson Martins (Position: 5) - **Central Midfielder:** Mario Rui (Position: 19) - **Central Midfielder:** Ricardo Quaresma (Position: 18) - **Central Midfielder:** Adrien Silva (Position: 23) - **Forwards:** - **Left Forward:** Rúben",6000.
"The image is a screenshot from the FIFA 19 video game, showing the starting lineup for Portugal in a match. The interface displays various players and their positions on the field. Here is a detailed breakdown of the infor

In [ ]:
print(clip_by_speech)
print(clip_by_caption)

                                               make_video
0  /home/razvantalexandru/.pixeltable/tmp/tmpryxexn6j.mp4


In [80]:
vtable = pxt.get_table("cache_cd64.table")

In [81]:
print(vtable)

Table
'cache_cd64.table'

    Column Name   Type                       Computed With
          video  Video                                    
  audio_extract  Audio  extract_audio(video, format='mp3')
